In [2]:
!pip install transformers gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 99.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 2.1 MB/s eta 0:00:00


In [14]:
import os
from huggingface_hub import HfApi, Repository, whoami, login

# Step 1: Login (you'll be prompted to paste your Hugging Face token)
login()

# Step 2: Config
SPACE_NAME = "dialo-chatbot2"
api = HfApi()
username = whoami()["name"]
full_repo_name = f"{username}/{SPACE_NAME}"

# Step 3: Create the Space if it doesn't exist
repo_url = api.create_repo(
    repo_id=full_repo_name,
    repo_type="space",
    space_sdk="gradio",
    exist_ok=True
)

# Step 4: Clone the empty repo into a fresh directory
LOCAL_DIR = "./dialo_space"
if os.path.exists(LOCAL_DIR):
    import shutil
    shutil.rmtree(LOCAL_DIR)  # clean it up first

repo = Repository(
    local_dir=LOCAL_DIR,
    clone_from=repo_url
)

# Step 5: Write final safe version of app.py
with open(os.path.join(LOCAL_DIR, "app.py"), "w") as f:
    f.write("""\
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import gradio as gr

# Load DialoGPT-small model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-small")
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-small")

# Final stable chat function (stateless)
def chat_with_bot(user_input, history):
    # Rebuild full conversation as plain text
    full_convo = ""
    for user, bot in history:
        full_convo += f"{user}{tokenizer.eos_token}{bot}{tokenizer.eos_token}"
    full_convo += f"{user_input}{tokenizer.eos_token}"

    input_ids = tokenizer.encode(full_convo, return_tensors="pt")

    output_ids = model.generate(
        input_ids,
        max_length=1000,
        pad_token_id=tokenizer.eos_token_id
    )

    reply = tokenizer.decode(
        output_ids[:, input_ids.shape[-1]:][0],
        skip_special_tokens=True
    )

    # Return new copy of history (don't mutate original)
    updated_history = history + [(user_input, reply)]
    return updated_history, updated_history

# Gradio chat interface
gr.ChatInterface(
    fn=chat_with_bot,
    title="💬 Chat with DialoGPT",
    description="A simple open-domain chatbot powered by Microsoft's DialoGPT-small model."
).launch()
""")

# Step 6: Write requirements.txt
with open(os.path.join(LOCAL_DIR, "requirements.txt"), "w") as f:
    f.write("transformers\ntorch\ngradio\n")

# Step 7: Push to Hugging Face
repo.push_to_hub(commit_message="🚀 Deploy chatbot")
print(f"🎉 Deployed! Your Space is live at: https://huggingface.co/spaces/{full_repo_name}")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
Cloning https://huggingface.co/spaces/zda23/dialo-chatbot2 into local empty directory.
To https://huggingface.co/spaces/zda23/dialo-chatbot2
   35f3d9f..e8eb2ec  main -> main

   35f3d9f..e8eb2ec  main -> main



🎉 Deployed! Your Space is live at: https://huggingface.co/spaces/zda23/dialo-chatbot2


In [15]:
import nbformat

# Replace with your actual filename
notebook_filename = "microsoftchatbot.ipynb"

# Load notebook
with open(notebook_filename, "r", encoding="utf-8") as f:
    nb = nbformat.read(f, as_version=4)

# Remove corrupted widget metadata
if "widgets" in nb["metadata"]:
    del nb["metadata"]["widgets"]

# Save the cleaned notebook
cleaned_name = "CLEANED_" + notebook_filename
with open(cleaned_name, "w", encoding="utf-8") as f:
    nbformat.write(nb, f)

print(f"✅ Cleaned notebook saved as: {cleaned_name}")

FileNotFoundError: [Errno 2] No such file or directory: 'microsoftchatbot.ipynb'

In [ ]:
from google.colab import files
files.download(cleaned_name)